In [1]:
import streamlit as st
import pandas as pd
import requests
import numpy as np
import yfinance as yf
import matplotlib
%matplotlib inline


In [2]:
import sys
print(sys.executable)


C:\Users\brand\new_env\Scripts\python.exe


## Cash Flows

In [29]:
cf_path = '../data/panamadao_cf.csv'
panamacf = pd.read_csv(cf_path)
panamacf.dropna(inplace=True)

panamacf['DAY'] = pd.to_datetime(panamacf['DAY'])
panamacf.set_index('DAY', inplace=True)
panamacf

,DAILY_REVENUE,DAILY_EXPENSE,NET_PROFIT
DAY,,,
2024-05-20,30.986428,30.986428,0.000000
2024-05-19,34.056852,34.056852,0.000000
2024-05-18,31.139696,31.139696,0.000000
2024-05-17,15.757103,15.181030,0.576073
2024-05-16,26.861555,13.430778,13.430778
...,...,...,...
2023-03-26,17.702967,0.000000,17.702967
2023-03-25,0.000000,0.000000,0.000000
2023-03-24,531.920194,0.000000,531.920194


## Assets

In [12]:
assets_path = '../data/asset_prices.csv'
assets = pd.read_csv(assets_path)
assets.dropna(inplace=True)
assets = assets[assets['DAY'] >= '2023-03-22']
assets['DAY'] = pd.to_datetime(assets['DAY'])

# Pivot the data to have symbols as columns
pivot_assets = assets.pivot(index='DAY', columns='SYMBOL', values='PRICE').reset_index()

# Display the pivoted DataFrame
pivot_assets.set_index('DAY', inplace=True)
pivot_assets.columns = [f'DAILY_PRICE_{col}' for col in pivot_assets.columns]
pivot_assets = pivot_assets.reset_index()
pivot_assets

,DAY,DAILY_PRICE_DPI,DAILY_PRICE_PAXG,DAILY_PRICE_RETH,DAILY_PRICE_SOL,DAILY_PRICE_WBTC,DAILY_PRICE_WSTETH
0,2023-03-22,84.457211,1994.171241,1943.193724,22.885376,28791.601013,2023.790378
1,2023-03-23,83.412666,2014.229622,1978.132073,26.028662,28754.493594,2058.390353
2,2023-03-24,82.238418,2010.110781,1952.633363,22.130990,28510.610009,2032.886739
3,2023-03-25,78.278926,2015.053186,1893.528757,21.123700,27881.394353,1968.846231
4,2023-03-26,78.453328,2011.649186,1924.050865,21.123650,28280.352719,2006.394028
...,...,...,...,...,...,...,...
417,2024-05-12,97.804624,2343.778463,3309.780000,147.294545,61564.660776,3426.778054
418,2024-05-13,97.481665,2351.684892,3298.731671,149.971232,63051.233477,3471.846182
419,2024-05-14,92.418617,2341.350000,3265.236141,148.850961,62889.537480,3436.445226
420,2024-05-15,96.340000,2378.070000,3340.980000,156.320000,65909.000000,3516.890000


## PanamaDAO Portfolio

In [13]:
panamadao_path = '../data/panamadao_returns.csv'
panamadao_returns = pd.read_csv(panamadao_path)
panamadao_returns.dropna(inplace=True)
panamadao_returns


,DAY,SYMBOL,TOTAL_FILLED_BALANCE,DAILY_PRICE,TOTAL_VALUE_IN_USD,TOTAL_USD_IN_TREASURY,COMPOSITION
0,2023-03-22 00:00:00.000,ETH,0.100000,1743.360000,174.336000,174.336000,1.000000
1,2023-03-22 00:00:00.000,USDC,0.000000,1.000000,0.000000,174.336000,0.000000
2,2023-03-22 00:00:00.000,WETH,0.000000,1743.360000,0.000000,174.336000,0.000000
3,2023-03-23 00:00:00.000,ETH,0.140000,1825.050000,255.507000,255.507000,1.000000
4,2023-03-23 00:00:00.000,USDC,0.000000,1.000000,0.000000,255.507000,0.000000
...,...,...,...,...,...,...,...
1258,2024-05-14 00:00:00.000,USDC,270.659535,0.999893,270.630574,32341.145506,0.008368
1259,2024-05-14 00:00:00.000,WETH,0.001900,2881.980000,5.475762,32341.145506,0.000169
1260,2024-05-15 00:00:00.000,ETH,11.230723,3021.480000,33933.404930,34209.805277,0.991920
1261,2024-05-15 00:00:00.000,USDC,270.659535,1.000000,270.659535,34209.805277,0.007912


In [15]:
# Convert 'DAY' to datetime
panamadao_returns['DAY'] = pd.to_datetime(panamadao_returns['DAY'])

# Sort by 'DAY' and 'SYMBOL'
panamadao_returns = panamadao_returns.sort_values(by=['DAY', 'SYMBOL'])

# Calculate previous day price
panamadao_returns['prev_day_price'] = panamadao_returns.groupby('SYMBOL')['DAILY_PRICE'].shift(1)

# Calculate daily return
panamadao_returns['daily_return'] = (panamadao_returns['DAILY_PRICE'] - panamadao_returns['prev_day_price']) / panamadao_returns['prev_day_price']

# Calculate weighted daily return
panamadao_returns['weighted_daily_return1'] = panamadao_returns['daily_return'] * panamadao_returns['COMPOSITION']

# Filter dates after '2023-07-01'
filtered_returns = panamadao_returns

# Calculate weighted daily return per day
weighted_daily_returns = filtered_returns.groupby('DAY').apply(lambda x: (x['daily_return'] * x['COMPOSITION']).sum()).reset_index(name='weighted_daily_return')

# Calculate cumulative returns
weighted_daily_returns['daily_factor'] = 1 + weighted_daily_returns['weighted_daily_return']
weighted_daily_returns['cumulative_return'] = (np.exp(np.log(weighted_daily_returns['daily_factor']).cumsum()) - 1)

# Calculate metrics
average_daily_return = weighted_daily_returns['weighted_daily_return'].mean()
stddev_daily_return = weighted_daily_returns['weighted_daily_return'].std()

# Calculate Sharpe ratio
annualized_return = average_daily_return * 365
annualized_risk = stddev_daily_return * np.sqrt(365)
sharpe_ratio = (annualized_return - 0.05) / annualized_risk

# Prepare base return
base_return = weighted_daily_returns[['DAY', 'cumulative_return']].copy()
base_return = base_return.dropna().rename(columns={'cumulative_return': 'base_cumulative_return'})

# Combine results
combined = base_return[['DAY', 'base_cumulative_return']].sort_values('DAY')

# Normalize returns
first_value = combined['base_cumulative_return'].iloc[0]
combined['PanamaDAO_treasury_return'] = 100 + (100 * (combined['base_cumulative_return'] - first_value))

# Final output
historical_normalized_returns = combined[['DAY', 'PanamaDAO_treasury_return']]
historical_normalized_returns.set_index('DAY', inplace=True)
print(historical_normalized_returns)

            PanamaDAO_treasury_return
DAY                                  
2023-03-22                 100.000000
2023-03-23                 104.685779
2023-03-24                 101.257342
2023-03-25                 100.418732
2023-03-26                 102.108572
...                               ...
2024-05-11                 166.906223
2024-05-12                 167.761691
2024-05-13                 168.638729
2024-05-14                 164.987403
2024-05-15                 172.910446

[421 rows x 1 columns]


C:\Users\brand\AppData\Local\Temp\ipykernel_8232\1426423555.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_daily_returns = filtered_returns.groupby('DAY').apply(lambda x: (x['daily_return'] * x['COMPOSITION']).sum()).reset_index(name='weighted_daily_return')


## Combining Asset and Portfolio Data

In [17]:
pivot_data = panamadao_returns.pivot(index='DAY', columns='SYMBOL', values=['TOTAL_FILLED_BALANCE', 'DAILY_PRICE', 'TOTAL_VALUE_IN_USD', 'COMPOSITION'])
pivot_data.columns = ['_'.join(col).strip() for col in pivot_data.columns.values]
pivot_data = pivot_data.reset_index()
pivot_data.set_index('DAY', inplace=True)


In [27]:
pivot_data

,TOTAL_FILLED_BALANCE_ETH,TOTAL_FILLED_BALANCE_USDC,TOTAL_FILLED_BALANCE_WETH,DAILY_PRICE_ETH,DAILY_PRICE_USDC,DAILY_PRICE_WETH,TOTAL_VALUE_IN_USD_ETH,TOTAL_VALUE_IN_USD_USDC,TOTAL_VALUE_IN_USD_WETH,COMPOSITION_ETH,COMPOSITION_USDC,COMPOSITION_WETH
DAY,,,,,,,,,,,,
2023-03-22,0.100000,0.000000,0.0000,1743.36,1.000000,1743.36,174.336000,0.000000,0.000000,1.000000,0.000000,0.000000
2023-03-23,0.140000,0.000000,0.0000,1825.05,1.000000,1825.05,255.507000,0.000000,0.000000,1.000000,0.000000,0.000000
2023-03-24,0.437000,0.000000,0.0000,1765.28,1.010000,1765.28,771.427360,0.000000,0.000000,1.000000,0.000000,0.000000
2023-03-25,0.437000,0.000000,0.0000,1750.66,1.010000,1750.66,765.038420,0.000000,0.000000,1.000000,0.000000,0.000000
2023-03-26,0.447000,0.000000,0.0000,1780.12,0.999827,1780.12,795.713640,0.000000,0.000000,1.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-11,11.047240,270.659535,0.0019,2915.79,1.000000,2915.79,32211.431920,270.659535,5.540001,0.991498,0.008331,0.000171
2024-05-12,11.062640,270.659535,0.0019,2930.86,0.999977,2930.86,32423.049070,270.653310,5.568634,0.991553,0.008277,0.000170
2024-05-13,11.090865,270.659535,0.0019,2946.31,0.999938,2946.31,32677.126458,270.642754,5.597989,0.991617,0.008213,0.000170


In [18]:
filtered_columns = [col for col in pivot_data.columns if 'COMPOSITION_' in col or 'DAILY_PRICE_' in col]
pivot_data_filtered = pivot_data[filtered_columns]
pivot_data_filtered


,DAILY_PRICE_ETH,DAILY_PRICE_USDC,DAILY_PRICE_WETH,COMPOSITION_ETH,COMPOSITION_USDC,COMPOSITION_WETH
DAY,,,,,,
2023-03-22,1743.36,1.000000,1743.36,1.000000,0.000000,0.000000
2023-03-23,1825.05,1.000000,1825.05,1.000000,0.000000,0.000000
2023-03-24,1765.28,1.010000,1765.28,1.000000,0.000000,0.000000
2023-03-25,1750.66,1.010000,1750.66,1.000000,0.000000,0.000000
2023-03-26,1780.12,0.999827,1780.12,1.000000,0.000000,0.000000
...,...,...,...,...,...,...
2024-05-11,2915.79,1.000000,2915.79,0.991498,0.008331,0.000171
2024-05-12,2930.86,0.999977,2930.86,0.991553,0.008277,0.000170
2024-05-13,2946.31,0.999938,2946.31,0.991617,0.008213,0.000170


In [19]:
# Convert assets DataFrame to wide format
assets_pivot = assets.pivot(index='DAY', columns='SYMBOL', values='PRICE')

# Rename columns to match the format in pivot_data_filtered
assets_pivot.columns = [f'DAILY_PRICE_{col}' for col in assets_pivot.columns]
assets_pivot.index

DatetimeIndex(['2023-03-22', '2023-03-23', '2023-03-24', '2023-03-25',
               '2023-03-26', '2023-03-27', '2023-03-28', '2023-03-29',
               '2023-03-30', '2023-03-31',
               ...
               '2024-05-07', '2024-05-08', '2024-05-09', '2024-05-10',
               '2024-05-11', '2024-05-12', '2024-05-13', '2024-05-14',
               '2024-05-15', '2024-05-16'],
              dtype='datetime64[ns]', name='DAY', length=422, freq=None)

In [20]:
pivot_data_filtered.index = pd.to_datetime(pivot_data_filtered.index)

In [22]:
combined_data = pivot_data_filtered.merge(assets_pivot, left_index=True, right_index=True, how='left')
combined_data

,DAILY_PRICE_ETH,DAILY_PRICE_USDC,DAILY_PRICE_WETH,COMPOSITION_ETH,COMPOSITION_USDC,COMPOSITION_WETH,DAILY_PRICE_DPI,DAILY_PRICE_PAXG,DAILY_PRICE_RETH,DAILY_PRICE_SOL,DAILY_PRICE_WBTC,DAILY_PRICE_WSTETH
DAY,,,,,,,,,,,,
2023-03-22,1743.36,1.000000,1743.36,1.000000,0.000000,0.000000,84.457211,1994.171241,1943.193724,22.885376,28791.601013,2023.790378
2023-03-23,1825.05,1.000000,1825.05,1.000000,0.000000,0.000000,83.412666,2014.229622,1978.132073,26.028662,28754.493594,2058.390353
2023-03-24,1765.28,1.010000,1765.28,1.000000,0.000000,0.000000,82.238418,2010.110781,1952.633363,22.130990,28510.610009,2032.886739
2023-03-25,1750.66,1.010000,1750.66,1.000000,0.000000,0.000000,78.278926,2015.053186,1893.528757,21.123700,27881.394353,1968.846231
2023-03-26,1780.12,0.999827,1780.12,1.000000,0.000000,0.000000,78.453328,2011.649186,1924.050865,21.123650,28280.352719,2006.394028
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-11,2915.79,1.000000,2915.79,0.991498,0.008331,0.000171,96.341420,2345.431329,3309.780000,146.651336,61185.560690,3411.258922
2024-05-12,2930.86,0.999977,2930.86,0.991553,0.008277,0.000170,97.804624,2343.778463,3309.780000,147.294545,61564.660776,3426.778054
2024-05-13,2946.31,0.999938,2946.31,0.991617,0.008213,0.000170,97.481665,2351.684892,3298.731671,149.971232,63051.233477,3471.846182


In [23]:
# Initialize the composition columns for new assets with zeros
for asset in assets['SYMBOL'].unique():
    comp_col = f'COMPOSITION_{asset}'
    if comp_col not in combined_data.columns:
        combined_data[comp_col] = 0.0

combined_data

,DAILY_PRICE_ETH,DAILY_PRICE_USDC,DAILY_PRICE_WETH,COMPOSITION_ETH,COMPOSITION_USDC,COMPOSITION_WETH,DAILY_PRICE_DPI,DAILY_PRICE_PAXG,DAILY_PRICE_RETH,DAILY_PRICE_SOL,DAILY_PRICE_WBTC,DAILY_PRICE_WSTETH,COMPOSITION_RETH,COMPOSITION_DPI,COMPOSITION_SOL,COMPOSITION_WSTETH,COMPOSITION_WBTC,COMPOSITION_PAXG
DAY,,,,,,,,,,,,,,,,,,
2023-03-22,1743.36,1.000000,1743.36,1.000000,0.000000,0.000000,84.457211,1994.171241,1943.193724,22.885376,28791.601013,2023.790378,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-23,1825.05,1.000000,1825.05,1.000000,0.000000,0.000000,83.412666,2014.229622,1978.132073,26.028662,28754.493594,2058.390353,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-24,1765.28,1.010000,1765.28,1.000000,0.000000,0.000000,82.238418,2010.110781,1952.633363,22.130990,28510.610009,2032.886739,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-25,1750.66,1.010000,1750.66,1.000000,0.000000,0.000000,78.278926,2015.053186,1893.528757,21.123700,27881.394353,1968.846231,0.0,0.0,0.0,0.0,0.0,0.0
2023-03-26,1780.12,0.999827,1780.12,1.000000,0.000000,0.000000,78.453328,2011.649186,1924.050865,21.123650,28280.352719,2006.394028,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-11,2915.79,1.000000,2915.79,0.991498,0.008331,0.000171,96.341420,2345.431329,3309.780000,146.651336,61185.560690,3411.258922,0.0,0.0,0.0,0.0,0.0,0.0
2024-05-12,2930.86,0.999977,2930.86,0.991553,0.008277,0.000170,97.804624,2343.778463,3309.780000,147.294545,61564.660776,3426.778054,0.0,0.0,0.0,0.0,0.0,0.0
2024-05-13,2946.31,0.999938,2946.31,0.991617,0.008213,0.000170,97.481665,2351.684892,3298.731671,149.971232,63051.233477,3471.846182,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
weighted_daily_returns.set_index('DAY', inplace=True)

In [25]:
historial_returns = weighted_daily_returns['weighted_daily_return']

def sortino(returns):
    # Calculate the downside risk
    target_return = 0  # This is typically set to zero
    downside_returns = returns[returns < target_return]
    downside_risk = np.sqrt(np.mean(downside_returns**2))

    # Calculate the expected return
    expected_return = returns.mean()

    # Annualize the expected return and downside risk
    trading_days_per_year = 365
    annualized_return = expected_return * trading_days_per_year
    annualized_downside_risk = downside_risk * np.sqrt(trading_days_per_year)

    # Compute the Sortino ratio
    sortino_ratio = (annualized_return - target_return) / annualized_downside_risk

    # Output the Sortino ratio
    print(f"Sortino Ratio: {sortino_ratio}")
    return sortino_ratio

sortino(historial_returns)

Sortino Ratio: 1.2634435305183658


1.2634435305183658

In [26]:
historical_cumulative_return = weighted_daily_returns['cumulative_return']


### MVO Rebalance

In [30]:
import pandas as pd

# Assuming pivot_data, pivot_assets, and panamacf are already loaded DataFrames

# Merge pivot_data and pivot_assets on DAY
combined_data = pd.merge(pivot_data, pivot_assets, on='DAY')

# Merge the combined_data with panamacf on DAY
combined_data = pd.merge(combined_data, panamacf, left_on='DAY', right_on='DAY')

# Ensure the data is sorted by DAY
combined_data = combined_data.sort_values('DAY').set_index('DAY')


In [31]:
combined_data

,TOTAL_FILLED_BALANCE_ETH,TOTAL_FILLED_BALANCE_USDC,TOTAL_FILLED_BALANCE_WETH,DAILY_PRICE_ETH,DAILY_PRICE_USDC,DAILY_PRICE_WETH,TOTAL_VALUE_IN_USD_ETH,TOTAL_VALUE_IN_USD_USDC,TOTAL_VALUE_IN_USD_WETH,COMPOSITION_ETH,...,COMPOSITION_WETH,DAILY_PRICE_DPI,DAILY_PRICE_PAXG,DAILY_PRICE_RETH,DAILY_PRICE_SOL,DAILY_PRICE_WBTC,DAILY_PRICE_WSTETH,DAILY_REVENUE,DAILY_EXPENSE,NET_PROFIT
DAY,,,,,,,,,,,,,,,,,,,,,
2023-03-22,0.100000,0.000000,0.0000,1743.36,1.000000,1743.36,174.336000,0.000000,0.000000,1.000000,...,0.000000,84.457211,1994.171241,1943.193724,22.885376,28791.601013,2023.790378,179.438208,0.000000,179.438208
2023-03-23,0.140000,0.000000,0.0000,1825.05,1.000000,1825.05,255.507000,0.000000,0.000000,1.000000,...,0.000000,83.412666,2014.229622,1978.132073,26.028662,28754.493594,2058.390353,71.386217,0.000000,71.386217
2023-03-24,0.437000,0.000000,0.0000,1765.28,1.010000,1765.28,771.427360,0.000000,0.000000,1.000000,...,0.000000,82.238418,2010.110781,1952.633363,22.130990,28510.610009,2032.886739,531.920194,0.000000,531.920194
2023-03-25,0.437000,0.000000,0.0000,1750.66,1.010000,1750.66,765.038420,0.000000,0.000000,1.000000,...,0.000000,78.278926,2015.053186,1893.528757,21.123700,27881.394353,1968.846231,0.000000,0.000000,0.000000
2023-03-26,0.447000,0.000000,0.0000,1780.12,0.999827,1780.12,795.713640,0.000000,0.000000,1.000000,...,0.000000,78.453328,2011.649186,1924.050865,21.123650,28280.352719,2006.394028,17.702967,0.000000,17.702967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-11,11.047240,270.659535,0.0019,2915.79,1.000000,2915.79,32211.431920,270.659535,5.540001,0.991498,...,0.000171,96.341420,2345.431329,3309.780000,146.651336,61185.560690,3411.258922,52.930142,52.930142,0.000000
2024-05-12,11.062640,270.659535,0.0019,2930.86,0.999977,2930.86,32423.049070,270.653310,5.568634,0.991553,...,0.000170,97.804624,2343.778463,3309.780000,147.294545,61564.660776,3426.778054,45.045687,45.045687,0.000000
2024-05-13,11.090865,270.659535,0.0019,2946.31,0.999938,2946.31,32677.126458,270.642754,5.597989,0.991617,...,0.000170,97.481665,2351.684892,3298.731671,149.971232,63051.233477,3471.846182,44.447619,41.506503,2.941116


In [32]:
initial_units = combined_data.iloc[0][['TOTAL_FILLED_BALANCE_ETH', 'TOTAL_FILLED_BALANCE_USDC', 'TOTAL_FILLED_BALANCE_WETH']].values
all_assets = ['ETH', 'USDC', 'WETH']


In [37]:
import cvxpy as cp


def calculate_sortino_ratio(returns):
    target_return = 0
    downside_deviation = np.sqrt(np.mean(np.minimum(0, returns - target_return) ** 2))
    if downside_deviation == 0:
        return 0
    sortino_ratio = np.mean(returns - target_return) / downside_deviation
    if np.isnan(sortino_ratio):
        return 0
    return sortino_ratio

def update_portfolio_value(data, unit_portfolio, net_profit):
    daily_values = data[[f'DAILY_PRICE_{asset}' for asset in all_assets]].multiply(unit_portfolio, axis=1).sum(axis=1)
    daily_values += net_profit
    return daily_values

def calculate_daily_returns(daily_values):
    daily_returns = daily_values.pct_change().dropna()
    return daily_returns

def calculate_cumulative_returns(daily_returns):
    cumulative_returns = (1 + daily_returns).cumprod() - 1
    return cumulative_returns
def mvo_sortino(returns, sortino_ratios):
    n = returns.shape[1]
    weights = cp.Variable(n)
    portfolio_return = returns @ weights
    sortino_matrix = np.tile(sortino_ratios.values.reshape(1, -1), (len(returns), 1))
    portfolio_risk = cp.norm(returns - cp.multiply(sortino_matrix, cp.reshape(weights, (1, n))), 'fro')
    objective = cp.Maximize(cp.sum(portfolio_return))
    
    constraints = [
        cp.sum(weights) == 1,
        weights >= 0,
        weights <= 0.3
    ]
    constraints.append(weights[0] >= 0.2)
    
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.CLARABEL)
    return weights.value

def rebalance_portfolio(data, all_assets, rebalancing_frequency=7):
    rebalanced_data = data.copy()
    unit_portfolio = initial_units.copy()
    
    for start in range(0, len(data), rebalancing_frequency):
        end = min(start + rebalancing_frequency, len(data))
        period_data = data.iloc[start:end]
        
        if start != 0:
            period_returns = data.iloc[start:end][[f'DAILY_PRICE_{asset}' for asset in all_assets]].pct_change().dropna()
            sortino_ratios = period_returns.apply(calculate_sortino_ratio)
            optimal_weights = mvo_sortino(period_returns.values, sortino_ratios)
            
            current_prices = data.iloc[start][[f'DAILY_PRICE_{asset}' for asset in all_assets]].values
            portfolio_value = np.dot(unit_portfolio, current_prices)
            unit_portfolio = optimal_weights * portfolio_value / current_prices
        
        for i in range(start, end):
            current_prices = data.iloc[i][[f'DAILY_PRICE_{asset}' for asset in all_assets]].values
            portfolio_value = np.dot(unit_portfolio, current_prices)
            rebalanced_data.iloc[i][[f'COMPOSITION_{asset}' for asset in all_assets]] = unit_portfolio * current_prices / portfolio_value
    
    return rebalanced_data


In [38]:
# Set rebalancing frequency (e.g., 1 for daily, 7 for weekly, 30 for monthly)
rebalancing_frequency = 7  # Change this to the desired frequency

# Perform rebalancing using MVO with Sortino ratio target
rebalanced_data = rebalance_portfolio(combined_data, all_assets, rebalancing_frequency=rebalancing_frequency)

# Calculate daily portfolio values and returns
daily_values = update_portfolio_value(rebalanced_data, initial_units, combined_data['NET_PROFIT'])
daily_returns = calculate_daily_returns(daily_values)

# Calculate cumulative return for the rebalanced data
cumulative_return = calculate_cumulative_returns(daily_returns)

# Prepare base return
base_return = cumulative_return.reset_index()
base_return.columns = ['DAY', 'base_cumulative_return']

# Normalize returns
first_value = base_return['base_cumulative_return'].iloc[0]
base_return['PanamaDAO_treasury_return'] = 100 + (100 * (base_return['base_cumulative_return'] - first_value))

# Final output
normalized_returns = base_return[['DAY', 'PanamaDAO_treasury_return']]
normalized_returns.set_index('DAY', inplace=True)

# Print the first few rows of normalized returns
print(normalized_returns.head())


C:\Users\brand\AppData\Local\Temp\ipykernel_8232\1496929361.py:65: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  rebalanced_data.iloc[i][[f'COMPOSITION_{asset}' for asset in all_assets]] = unit_portfolio * current_prices / portfolio_value
C:

TypeError: unsupported operand type(s) for *: 'NoneType' and 'float'

In [49]:
import cvxpy as cp
import numpy as np
import pandas as pd

# Define all_assets including additional assets
all_assets = ['ETH', 'USDC', 'WETH', 'DPI', 'PAXG', 'RETH', 'SOL', 'WBTC', 'WSTETH']

# Initialize missing balance columns with default values if they are not present in the dataset
for asset in all_assets:
    balance_col = f'TOTAL_FILLED_BALANCE_{asset}'
    if balance_col not in combined_data.columns:
        combined_data[balance_col] = 0.0

# Initialize missing composition columns with default values if they are not present in the dataset
for asset in all_assets:
    composition_col = f'COMPOSITION_{asset}'
    if composition_col not in combined_data.columns:
        combined_data[composition_col] = 0.0

# Initial units are derived from the initial balances
initial_units = combined_data.iloc[0][[f'TOTAL_FILLED_BALANCE_{asset}' for asset in all_assets]].values

def calculate_sortino_ratio(returns):
    target_return = 0
    downside_deviation = np.sqrt(np.mean(np.minimum(0, returns - target_return) ** 2))
    if downside_deviation == 0:
        return 0
    sortino_ratio = np.mean(returns - target_return) / downside_deviation
    if np.isnan(sortino_ratio):
        return 0
    return sortino_ratio

def update_portfolio_value(data, unit_portfolio, net_profit):
    daily_values = data[[f'DAILY_PRICE_{asset}' for asset in all_assets]].multiply(unit_portfolio, axis=1).sum(axis=1)
    daily_values += net_profit
    return daily_values

def calculate_daily_returns(daily_values):
    daily_returns = daily_values.pct_change().dropna()
    return daily_returns

def calculate_cumulative_returns(daily_returns):
    cumulative_returns = (1 + daily_returns).cumprod() - 1
    return cumulative_returns

def mvo_sortino(returns, sortino_ratios):
    n = returns.shape[1]
    weights = cp.Variable(n)
    portfolio_return = returns @ weights
    sortino_matrix = np.tile(sortino_ratios.values.reshape(1, -1), (len(returns), 1))
    portfolio_risk = cp.norm(returns - cp.multiply(sortino_matrix, cp.reshape(weights, (1, n))), 'fro')
    objective = cp.Maximize(cp.sum(portfolio_return))
    
    constraints = [
        cp.sum(weights) == 1,
        weights >= 0,
        weights <= 0.3
    ]
    constraints.append(weights[0] >= 0.2)
    
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.CLARABEL)
    if problem.status not in ["optimal", "optimal_inaccurate"]:
        print(f"Optimization failed: {problem.status}")
        return None
    return weights.value

def rebalance_portfolio(data, all_assets, rebalancing_frequency=7):
    rebalanced_data = data.copy()
    unit_portfolio = initial_units.copy()
    
    for start in range(0, len(data), rebalancing_frequency):
        end = min(start + rebalancing_frequency, len(data))
        period_data = data.iloc[start:end]
        
        if start != 0:
            # Apply net profit/loss to the unit portfolio
            for i in range(start, end):
                current_prices = data.iloc[i][[f'DAILY_PRICE_{asset}' for asset in all_assets]].values
                net_profit = data.iloc[i]['NET_PROFIT']
                total_value = np.dot(unit_portfolio, current_prices)
                unit_portfolio += (net_profit / total_value) * (unit_portfolio / current_prices)
            
            # Perform rebalancing
            period_returns = data.iloc[start:end][[f'DAILY_PRICE_{asset}' for asset in all_assets]].pct_change().dropna()
            if period_returns.shape[0] == 0 or period_returns.shape[1] == 0:
                print(f"Skipping period {start} to {end} due to insufficient data")
                continue
            
            sortino_ratios = period_returns.apply(calculate_sortino_ratio)
            optimal_weights = mvo_sortino(period_returns.values, sortino_ratios)
            
            if optimal_weights is None:
                print(f"Skipping period {start} to {end} due to optimization failure")
                continue
            
            current_prices = data.iloc[start][[f'DAILY_PRICE_{asset}' for asset in all_assets]].values
            portfolio_value = np.dot(unit_portfolio, current_prices)
            unit_portfolio = optimal_weights * portfolio_value / current_prices
            
            print(f"Optimal weights at period {start} to {end}: {optimal_weights}")
            print(f"Updated unit portfolio: {unit_portfolio}")
        
        for i in range(start, end):
            current_prices = data.iloc[i][[f'DAILY_PRICE_{asset}' for asset in all_assets]].values
            portfolio_value = np.dot(unit_portfolio, current_prices)
            rebalanced_data.loc[data.index[i], [f'COMPOSITION_{asset}' for asset in all_assets]] = unit_portfolio * current_prices / portfolio_value
    
    return rebalanced_data

# Set rebalancing frequency (e.g., 1 for daily, 7 for weekly, 30 for monthly)
rebalancing_frequency = 7  # Change this to the desired frequency

# Perform rebalancing using MVO with Sortino ratio target
rebalanced_data = rebalance_portfolio(combined_data, all_assets, rebalancing_frequency=rebalancing_frequency)

# Calculate daily portfolio values and returns
daily_values = update_portfolio_value(rebalanced_data, initial_units, combined_data['NET_PROFIT'])
daily_returns = calculate_daily_returns(daily_values)

# Calculate cumulative return for the rebalanced data
cumulative_return = calculate_cumulative_returns(daily_returns)

# Prepare base return
base_return = cumulative_return.reset_index()
base_return.columns = ['DAY', 'base_cumulative_return']

# Normalize returns
first_value = base_return['base_cumulative_return'].iloc[0]
base_return['PanamaDAO_treasury_return'] = 100 + (100 * (base_return['base_cumulative_return'] - first_value))

# Final output
normalized_returns = base_return[['DAY', 'PanamaDAO_treasury_return']]
normalized_returns.set_index('DAY', inplace=True)

# Print the first few rows of normalized returns
print(normalized_returns.head())


Optimal weights at period 7 to 14: [2.99999984e-01 2.72861818e-09 2.99999962e-01 1.52416215e-08
 1.02208774e-08 2.99999961e-01 2.91642285e-09 2.20534587e-09
 1.00000060e-01]
Updated unit portfolio: [3.16843279e-02 5.17847674e-07 3.16843256e-02 3.63799162e-08
 9.73773072e-10 2.92195018e-02 2.59740920e-08 1.46484988e-11
 9.34290837e-03]
Optimal weights at period 14 to 21: [2.00000003e-01 1.26643605e-08 7.69894035e-10 8.50171531e-10
 7.37958226e-10 1.99999955e-01 2.99999999e-01 2.99999997e-01
 3.05124765e-08]
Updated unit portfolio: [2.10234200e-02 2.54322909e-06 8.09290268e-11 2.03234719e-09
 7.23594471e-11 1.96501059e-02 2.84531873e+00 2.09973202e-03
 2.86215881e-09]
Optimal weights at period 21 to 28: [2.47966357e-01 3.63249668e-10 1.52033654e-01 5.13249901e-09
 3.56445043e-10 2.99999991e-01 5.57475422e-10 3.66406064e-10
 2.99999992e-01]
Updated unit portfolio: [2.74910344e-02 7.75179537e-08 1.68553607e-02 1.33773471e-08
 3.74383172e-11 3.11633130e-02 4.94835699e-09 2.58396795e-12
 2.9

In [52]:
cumulative_return

DAY
2023-03-23   -0.282335
2023-03-24    1.002543
2023-03-25   -0.505148
2023-03-26   -0.446780
2023-03-27   -0.406423
                ...   
2024-05-11   -0.175805
2024-05-12   -0.171545
2024-05-13   -0.158864
2024-05-14   -0.150961
2024-05-15   -0.028907
Length: 420, dtype: float64

In [53]:
normalized_returns

,PanamaDAO_treasury_return
DAY,
2023-03-23,100.000000
2023-03-24,228.487879
2023-03-25,77.718778
2023-03-26,83.555542
2023-03-27,87.591249
...,...
2024-05-11,110.653061
2024-05-12,111.079039
2024-05-13,112.347113
